<a href="https://colab.research.google.com/github/pathbit/pathbit-academy-ai/blob/master/0002_embeddings_vetorizacao/notebooks/embeddings_vetorizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ✨ **Pathbit Academy AI**
---

## 🎯 **Artigo 0002: Embeddings e Vetorização - Como a IA Entende Texto**

🚨 **IMPORTANTE:**

*💥 QUALQUER PESSOA QUE CONSIGA RESOLVER A EQUAÇÃO `2 + 2 = ?` PODE CONTINUAR OS PASSOS ABAIXO*

**Artigo de referência:** [https://github.com/pathbit/pathbit-academy-ai/blob/master/0002_embeddings_vetorizacao/article/ARTICLE.md](https://github.com/pathbit/pathbit-academy-ai/blob/master/0002_embeddings_vetorizacao/article/ARTICLE.md)

**Artigo anterior:** [https://github.com/pathbit/pathbit-academy-ai/blob/master/0001_llm_x_lrm/article/ARTICLE.md](https://github.com/pathbit/pathbit-academy-ai/blob/master/0001_llm_x_lrm/article/ARTICLE.md)

---

## 🎯 **Este notebook contém TUDO que você precisa:**
- ✅ **Instalação automática** de dependências
- ✅ **Configuração** de modelos
- ✅ **9 exemplos práticos** de embeddings
- ✅ **Código pronto para usar** - sem arquivos externos
- ✅ **Funciona no Google Colab** e localmente


### 🔧 **Correção automática para Google Colab**

🚨 **IMPORTANTE:** Se você estiver executando no Google Colab, esta célula corrige automaticamente problemas de compatibilidade do `tqdm`.


In [1]:
# 🔧 CORREÇÃO AUTOMÁTICA PARA GOOGLE COLAB
# ==========================================
# Esta célula resolve automaticamente conflitos de dependências do tqdm

# Detectar se estamos no Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Detectado: Google Colab")
    print("🔧 Aplicando correção para conflito de tqdm...")
    
    # CORREÇÃO: Atualizar tqdm para resolver conflitos de dependências
    get_ipython().run_line_magic('pip', 'install --upgrade tqdm>=4.67 --force-reinstall --quiet')
    print("✅ tqdm atualizado com sucesso!")
    print("📦 Versão do tqdm corrigida para resolver conflitos com datasets e dataproc-spark-connect")
    
except ImportError:
    IN_COLAB = False
    print("💻 Detectado: Ambiente Local")
    print("ℹ️  Correção do tqdm não necessária no ambiente local")

print("\n🎯 Ambiente configurado! Continue com a próxima célula.")


💻 Detectado: Ambiente Local
ℹ️  Correção do tqdm não necessária no ambiente local

🎯 Ambiente configurado! Continue com a próxima célula.


In [2]:
# 🔧 CONFIGURAÇÃO INICIAL DO AMBIENTE
# ======================================

# 🔇 Suprimir avisos ANTES de qualquer importação
import warnings
import os
import sys

# Suprimir avisos específicos
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Tentar suprimir aviso do tqdm (se disponível)
try:
    from tqdm import TqdmExperimentalWarning
    warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)
except ImportError:
    pass

# Configurar ambiente para evitar problemas de importação
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

print("🔧 Ambiente configurado com sucesso!")

🔧 Ambiente configurado com sucesso!


### ֎ **O que são Embeddings?**

**Embeddings** são representações numéricas de texto que capturam o significado semântico. É como transformar "cachorro" e "animal de estimação" em números que ficam próximos no espaço matemático, mesmo sendo palavras diferentes.

![Conceito de Embeddings](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/01.png)

**Vetorização** é o processo de converter texto em esses números. Não é mágica, é matemática aplicada com muito texto e poder computacional.

![Processo de Vetorização](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/02.png)

#### ⁉️ Por que são importantes?

- **Busca Semântica:** Encontrar documentos similares baseado no significado
- **Classificação:** Categorizar documentos automaticamente  
- **Recomendação:** Sugerir conteúdo similar ao que o usuário gosta
- **RAG:** Base para sistemas de Retrieval Augmented Generation


In [3]:
# 📦 INSTALAÇÃO DAS DEPENDÊNCIAS
# ================================
# Instalação das bibliotecas necessárias para trabalhar com embeddings

# Instalar dependências (Colab precisa do %pip)
if IN_COLAB:
    print("📦 Instalando dependências no Google Colab...")
    get_ipython().run_line_magic('pip', 'install -q sentence-transformers scikit-learn matplotlib seaborn pandas plotly tqdm>=4.67')
else:
    print("📦 Verificando dependências no ambiente local...")
    try:
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        import seaborn as sns
        from sentence_transformers import SentenceTransformer
        from sklearn.metrics.pairwise import cosine_similarity
        from sklearn.cluster import KMeans
        from sklearn.decomposition import PCA
        print("✅ Todas as dependências já estão instaladas!")
    except ImportError as e:
        print(f"⚠️ Instalando dependências faltantes: {e}")
        import subprocess
        import sys
        
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "-q",
            "sentence-transformers", "scikit-learn", "matplotlib", 
            "seaborn", "pandas", "plotly", "tqdm>=4.67"
        ])

# Importações principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Configurações adicionais
plt.style.use('default')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("🎯 Notebook pronto para usar!")
print("📊 Dependências carregadas com sucesso!")


📦 Verificando dependências no ambiente local...
✅ Todas as dependências já estão instaladas!
🎯 Notebook pronto para usar!
📊 Dependências carregadas com sucesso!


In [4]:
# 🎨 FUNÇÃO DE FORMATAÇÃO VISUAL
# ===============================
# Adiciona formatação especial para destacar resultados importantes

def print_resultado_destacado(titulo, conteudo, tipo="info"):
    """
    Exibe resultados com formatação especial e visual atrativa.
    
    Args:
        titulo (str): Título do resultado
        conteudo (str): Conteúdo a ser exibido
        tipo (str): Tipo de resultado ("info", "success", "warning", "error")
    """
    # Cores e símbolos baseados no tipo
    cores = {
        "info": {"cor": "🔵", "emoji": "📊", "borda": "=", "cor_ascii": "🔷"},
        "success": {"cor": "🟢", "emoji": "✅", "borda": "=", "cor_ascii": "🔷"},
        "warning": {"cor": "🟡", "emoji": "⚠️", "borda": "-", "cor_ascii": "🔶"},
        "error": {"cor": "🔴", "emoji": "❌", "borda": "!", "cor_ascii": "🔴"}
    }
    
    config = cores.get(tipo, cores["info"])
    
    # Criar formatação visual
    largura = 70
    borda = config["borda"] * largura
    
    print(f"\n{config['emoji']} {borda}")
    print(f"{config['cor']} {titulo.center(largura-4)} {config['cor']}")
    print(f"{config['emoji']} {borda}")
    print(f"\n{conteudo}")
    print(f"\n{config['emoji']} {borda}")
    print(f"{config['cor']} {'🎯 RESULTADO CONCLUÍDO 🎯'.center(largura-4)} {config['cor']}")
    print(f"{config['emoji']} {borda}\n")

# Testar a formatação
print_resultado_destacado("TESTE DE FORMATAÇÃO", "Esta é uma demonstração da nova formatação visual que será usada em todos os resultados importantes do notebook!", "success")



✅ ======================================================================
🟢                        TESTE DE FORMATAÇÃO                         🟢
✅ ======================================================================

Esta é uma demonstração da nova formatação visual que será usada em todos os resultados importantes do notebook!

✅ ======================================================================
🟢                      🎯 RESULTADO CONCLUÍDO 🎯                       🟢
✅ ======================================================================



### 🛠️ **Classes e Funções Utilitárias**

Aqui estão todas as funções que você precisa para trabalhar com embeddings. **Não precisa baixar arquivos externos!**


In [5]:
# 🛠️ Classes e Funções para Embeddings
# ======================================

class EmbeddingsHelper:
    """
    Classe helper para trabalhar com embeddings de forma simples.
    Contém todas as funções necessárias para os exemplos.
    """
    
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """Inicializa o helper com um modelo de embeddings."""
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Carrega o modelo de embeddings."""
        try:
            print(f"🔄 Carregando modelo {self.model_name}...")
            self.model = SentenceTransformer(self.model_name)
            print(f"✅ Modelo {self.model_name} carregado com sucesso!")
            print(f"📊 Dimensões do embedding: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"❌ Erro ao carregar modelo: {e}")
            self.model = None
    
    def get_embedding(self, text):
        """Gera embedding para um texto."""
        if self.model is None:
            return None
        try:
            return self.model.encode([text])[0]
        except Exception as e:
            print(f"❌ Erro ao gerar embedding: {e}")
            return None
    
    def get_embeddings(self, texts):
        """Gera embeddings para uma lista de textos."""
        if self.model is None:
            return None
        try:
            return self.model.encode(texts)
        except Exception as e:
            print(f"❌ Erro ao gerar embeddings: {e}")
            return None
    
    def calculate_similarity(self, text1, text2):
        """Calcula similaridade coseno entre dois textos."""
        emb1 = self.get_embedding(text1)
        emb2 = self.get_embedding(text2)
        
        if emb1 is None or emb2 is None:
            return None
        
        similarity = cosine_similarity([emb1], [emb2])[0][0]
        return similarity
    
    def search_documents(self, query, documents, top_k=5):
        """Busca documentos mais similares a uma consulta."""
        if self.model is None:
            return []
        
        try:
            # Gerar embedding da consulta
            query_embedding = self.get_embedding(query)
            if query_embedding is None:
                return []
            
            # Gerar embeddings dos documentos
            doc_embeddings = self.get_embeddings(documents)
            if doc_embeddings is None:
                return []
            
            # Calcular similaridades
            similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
            
            # Ordenar por similaridade
            indices_ordenados = np.argsort(similarities)[::-1]
            
            # Retornar top_k resultados
            results = []
            for i in range(min(top_k, len(documents))):
                idx = indices_ordenados[i]
                results.append({
                    'documento': documents[idx],
                    'similaridade': similarities[idx],
                    'posicao': i + 1
                })
            
            return results
        except Exception as e:
            print(f"❌ Erro na busca: {e}")
            return []
    
    def cluster_documents(self, documents, n_clusters=3):
        """Agrupa documentos em clusters baseado em similaridade."""
        if self.model is None:
            return {}
        
        try:
            # Gerar embeddings
            embeddings = self.get_embeddings(documents)
            if embeddings is None:
                return {}
            
            # Aplicar K-Means
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            cluster_labels = kmeans.fit_predict(embeddings)
            
            # Organizar por clusters
            clusters = {}
            for i, label in enumerate(cluster_labels):
                if label not in clusters:
                    clusters[label] = []
                clusters[label].append({
                    'documento': documents[i],
                    'indice': i
                })
            
            return clusters
        except Exception as e:
            print(f"❌ Erro no clustering: {e}")
            return {}
    
    def detect_duplicates(self, documents, threshold=0.8):
        """Detecta documentos duplicados baseado em similaridade."""
        if self.model is None:
            return []
        
        try:
            # Gerar embeddings
            embeddings = self.get_embeddings(documents)
            if embeddings is None:
                return []
            
            # Calcular matriz de similaridade
            similarity_matrix = cosine_similarity(embeddings)
            
            # Encontrar pares duplicados
            duplicates = []
            for i in range(len(documents)):
                for j in range(i + 1, len(documents)):
                    similarity = similarity_matrix[i][j]
                    if similarity >= threshold:
                        duplicates.append({
                            'doc1': documents[i],
                            'doc2': documents[j],
                            'similarity': similarity,
                            'index1': i,
                            'index2': j
                        })
            
            return duplicates
        except Exception as e:
            print(f"❌ Erro na detecção de duplicatas: {e}")
            return []
    
    def visualize_embeddings(self, texts, title="Visualização de Embeddings"):
        """Visualiza embeddings em 2D usando PCA."""
        if self.model is None:
            return
        
        try:
            # Gerar embeddings
            embeddings = self.get_embeddings(texts)
            if embeddings is None:
                return
            
            # Reduzir dimensionalidade para 2D
            pca = PCA(n_components=2)
            embeddings_2d = pca.fit_transform(embeddings)
            
            # Criar gráfico
            plt.figure(figsize=(12, 8))
            
            # Plotar pontos
            plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                       alpha=0.7, s=100, c=range(len(texts)), cmap='viridis')
            
            # Adicionar labels
            for i, text in enumerate(texts):
                plt.annotate(f"{i+1}. {text[:30]}...", 
                            (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                            fontsize=9, alpha=0.8,
                            xytext=(5, 5), textcoords='offset points')
            
            plt.title(title, fontsize=14, fontweight='bold')
            plt.xlabel("Componente Principal 1", fontsize=12)
            plt.ylabel("Componente Principal 2", fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # Mostrar variância explicada
            var_explicada = pca.explained_variance_ratio_
            print(f"📊 Variância explicada:")
            print(f"   PC1: {var_explicada[0]:.1%}")
            print(f"   PC2: {var_explicada[1]:.1%}")
            print(f"   Total: {sum(var_explicada):.1%}")
            
        except Exception as e:
            print(f"❌ Erro na visualização: {e}")

# 🎯 Inicializar o helper
print("🚀 Inicializando EmbeddingsHelper...")
helper = EmbeddingsHelper('all-MiniLM-L6-v2')
print("✅ Helper pronto para uso!")


🚀 Inicializando EmbeddingsHelper...
🔄 Carregando modelo all-MiniLM-L6-v2...
✅ Modelo all-MiniLM-L6-v2 carregado com sucesso!
📊 Dimensões do embedding: 384
✅ Helper pronto para uso!


### 🎯 **Exemplo 1: Geração de Embeddings Básica**

![Transformação Texto para Números](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/03.png)


In [6]:
# 📝 Textos de exemplo
textos = [
    "O cachorro está brincando no parque",
    "O gato dorme no sofá", 
    "O carro está na garagem",
    "O animal de estimação está feliz",
    "O veículo precisa de gasolina"
]

print("📝 Textos de exemplo:")
for i, texto in enumerate(textos, 1):
    print(f"{i}. {texto}")

# 🔄 Gerar embeddings usando o helper
print("\n🔄 Gerando embeddings...")
embeddings = helper.get_embeddings(textos)

if embeddings is not None:
    # Formatação destacada para os resultados
    resultado_embeddings = f"""📝 TEXTOS PROCESSADOS:
{chr(10).join([f"• {i+1}. {texto}" for i, texto in enumerate(textos)])}

📊 INFORMAÇÕES TÉCNICAS:
• Forma dos embeddings: {embeddings.shape}
• Dimensões por texto: {embeddings.shape[1]}
• Total de textos processados: {embeddings.shape[0]}

🎯 O QUE ACONTECEU:
Cada texto foi convertido em um vetor de {embeddings.shape[1]} números que representam
seu significado semântico. Textos similares terão vetores próximos no espaço matemático.

💡 PRÓXIMO PASSO:
Agora podemos calcular similaridades e fazer buscas semânticas!"""

    print_resultado_destacado("EXEMPLO 1: GERAÇÃO DE EMBEDDINGS", resultado_embeddings, "success")
else:
    print_resultado_destacado("ERRO NA GERAÇÃO DE EMBEDDINGS", "Falha ao gerar embeddings. Verifique se o modelo foi carregado corretamente.", "error")


📝 Textos de exemplo:
1. O cachorro está brincando no parque
2. O gato dorme no sofá
3. O carro está na garagem
4. O animal de estimação está feliz
5. O veículo precisa de gasolina

🔄 Gerando embeddings...

✅ ======================================================================
🟢                  EXEMPLO 1: GERAÇÃO DE EMBEDDINGS                  🟢
✅ ======================================================================

📝 TEXTOS PROCESSADOS:
• 1. O cachorro está brincando no parque
• 2. O gato dorme no sofá
• 3. O carro está na garagem
• 4. O animal de estimação está feliz
• 5. O veículo precisa de gasolina

📊 INFORMAÇÕES TÉCNICAS:
• Forma dos embeddings: (5, 384)
• Dimensões por texto: 384
• Total de textos processados: 5

🎯 O QUE ACONTECEU:
Cada texto foi convertido em um vetor de 384 números que representam
seu significado semântico. Textos similares terão vetores próximos no espaço matemático.

💡 PRÓXIMO PASSO:
Agora podemos calcular similaridades e fazer buscas semânticas!

✅ ====

### 🎯 **Exemplo 2: Cálculo de Similaridade**

![Cálculo de Similaridade](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/04.png)


In [7]:
# 🔍 Calcular similaridade entre textos
texto1 = "O cachorro está brincando no parque"
texto2 = "O animal de estimação está feliz"
texto3 = "O carro está na garagem"

print("🔍 Teste de Similaridade:")
print(f"Texto 1: {texto1}")
print(f"Texto 2: {texto2}")
print(f"Texto 3: {texto3}")

sim_1_2 = helper.calculate_similarity(texto1, texto2)
sim_1_3 = helper.calculate_similarity(texto1, texto3)
sim_2_3 = helper.calculate_similarity(texto2, texto3)

# Formatação detalhada dos resultados
resultado_similaridade = f"""📝 TEXTOS ANALISADOS:
• Texto 1: "{texto1}"
• Texto 2: "{texto2}"
• Texto 3: "{texto3}"

📊 RESULTADOS DE SIMILARIDADE:
• Similaridade 1-2: {sim_1_2:.3f} (cachorro/animal estimação)
• Similaridade 1-3: {sim_1_3:.3f} (cachorro/carro)
• Similaridade 2-3: {sim_2_3:.3f} (animal estimação/carro)

🎯 ANÁLISE SEMÂNTICA:
• Textos 1-2: {"✅ MUITO SIMILARES" if sim_1_2 > 0.7 else "⚠️ MODERADAMENTE SIMILARES" if sim_1_2 > 0.5 else "❌ POUCO SIMILARES"} (ambos sobre animais)
• Textos 1-3: {"✅ DIFERENTES" if sim_1_3 < 0.3 else "⚠️ PARCIALMENTE SIMILARES" if sim_1_3 < 0.6 else "❌ MAIS SIMILARES QUE ESPERADO"} (animal vs veículo)
• Textos 2-3: {"✅ DIFERENTES" if sim_2_3 < 0.3 else "⚠️ PARCIALMENTE SIMILARES" if sim_2_3 < 0.6 else "❌ MAIS SIMILARES QUE ESPERADO"} (animal vs veículo)

💡 CONCLUSÃO:
A IA analisa a semântica dos textos e identifica relações conceituais.
Mesmo com palavras diferentes, consegue entender o contexto geral."""

print_resultado_destacado("EXEMPLO 2: CÁLCULO DE SIMILARIDADE", resultado_similaridade, "success")


🔍 Teste de Similaridade:
Texto 1: O cachorro está brincando no parque
Texto 2: O animal de estimação está feliz
Texto 3: O carro está na garagem

✅ ======================================================================
🟢                 EXEMPLO 2: CÁLCULO DE SIMILARIDADE                 🟢
✅ ======================================================================

📝 TEXTOS ANALISADOS:
• Texto 1: "O cachorro está brincando no parque"
• Texto 2: "O animal de estimação está feliz"
• Texto 3: "O carro está na garagem"

📊 RESULTADOS DE SIMILARIDADE:
• Similaridade 1-2: 0.290 (cachorro/animal estimação)
• Similaridade 1-3: 0.517 (cachorro/carro)
• Similaridade 2-3: 0.349 (animal estimação/carro)

🎯 ANÁLISE SEMÂNTICA:
• Textos 1-2: ❌ POUCO SIMILARES (ambos sobre animais)
• Textos 1-3: ⚠️ PARCIALMENTE SIMILARES (animal vs veículo)
• Textos 2-3: ⚠️ PARCIALMENTE SIMILARES (animal vs veículo)

💡 CONCLUSÃO:
A IA analisa a semântica dos textos e identifica relações conceituais.
Mesmo com palavras dife

### 🎯 **Exemplo 3: Busca Semântica**

![Busca Semântica](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/05.png)


In [8]:
# 🔍 Busca semântica em documentos
documentos = [
    "Python é uma linguagem de programação versátil e fácil de aprender",
    "Machine learning é uma área da inteligência artificial",
    "Data science combina estatística, programação e conhecimento de domínio",
    "Deep learning usa redes neurais para resolver problemas complexos",
    "Pandas é uma biblioteca Python para análise de dados",
    "Scikit-learn oferece ferramentas para machine learning",
    "Jupyter Notebook é ideal para análise exploratória de dados",
    "NumPy fornece arrays multidimensionais eficientes"
]

consulta = "ferramentas para análise de dados"

print("🔍 Busca Semântica:")
print(f"Consulta: '{consulta}'")
print("\n📚 Documentos disponíveis:")
for i, doc in enumerate(documentos, 1):
    print(f"{i}. {doc}")

print(f"\n🔄 Buscando documentos similares...")
resultados = helper.search_documents(consulta, documentos, top_k=3)

# Formatação detalhada dos resultados
resultado_busca = f"""🔍 CONSULTA REALIZADA:
"{consulta}"

📊 TOP 3 DOCUMENTOS MAIS SIMILARES:
"""
for i, resultado in enumerate(resultados, 1):
    resultado_busca += f"""
{i}. 🎯 POSIÇÃO: {resultado['posicao']} | SIMILARIDADE: {resultado['similaridade']:.3f}
   📄 DOCUMENTO: {resultado['documento']}
   {"✅ EXCELENTE MATCH" if resultado['similaridade'] > 0.7 else "⚠️ BOM MATCH" if resultado['similaridade'] > 0.5 else "❌ MATCH FRACO"}
"""

resultado_busca += f"""
💡 ANÁLISE DOS RESULTADOS:
• O sistema encontrou documentos relacionados a "análise de dados"
• Mesmo sem usar as palavras exatas, a IA entende o contexto semântico
• Pandas e Jupyter são ferramentas específicas para análise de dados
• Data science é o campo que mais se relaciona com a consulta

🎯 CONCLUSÃO:
A busca semântica permite encontrar conteúdo relevante mesmo quando
as palavras exatas não estão presentes no texto."""

print_resultado_destacado("EXEMPLO 3: BUSCA SEMÂNTICA", resultado_busca, "success")


🔍 Busca Semântica:
Consulta: 'ferramentas para análise de dados'

📚 Documentos disponíveis:
1. Python é uma linguagem de programação versátil e fácil de aprender
2. Machine learning é uma área da inteligência artificial
3. Data science combina estatística, programação e conhecimento de domínio
4. Deep learning usa redes neurais para resolver problemas complexos
5. Pandas é uma biblioteca Python para análise de dados
6. Scikit-learn oferece ferramentas para machine learning
7. Jupyter Notebook é ideal para análise exploratória de dados
8. NumPy fornece arrays multidimensionais eficientes

🔄 Buscando documentos similares...

✅ ======================================================================
🟢                     EXEMPLO 3: BUSCA SEMÂNTICA                     🟢
✅ ======================================================================

🔍 CONSULTA REALIZADA:
"ferramentas para análise de dados"

📊 TOP 3 DOCUMENTOS MAIS SIMILARES:

1. 🎯 POSIÇÃO: 1 | SIMILARIDADE: 0.576
   📄 DOCUMENTO: Ju

### 🎯 **Exemplo 4: Sistema de Recomendação**

![Sistema de Recomendação](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/08.png)


In [9]:
# 🎯 Sistema de recomendação baseado em embeddings
artigos = [
    "Como aprender Python do zero",
    "Introdução ao machine learning",
    "Data science com pandas",
    "Tutorial de deep learning",
    "Como fazer análise de dados",
    "Programação orientada a objetos",
    "Estatística para data science",
    "Visualização de dados com matplotlib"
]

# Simular histórico do usuário
historico_usuario = [
    "Como aprender Python do zero",
    "Data science com pandas"
]

print("🎯 Sistema de Recomendação:")
print("\n📚 Histórico do usuário:")
for i, artigo in enumerate(historico_usuario, 1):
    print(f"{i}. {artigo}")

print("\n🔄 Gerando recomendações...")

# Gerar embedding do histórico (média dos embeddings)
embeddings_historico = helper.get_embeddings(historico_usuario)
embedding_medio = np.mean(embeddings_historico, axis=0)

# Buscar artigos similares
embeddings_artigos = helper.get_embeddings(artigos)
similaridades = cosine_similarity([embedding_medio], embeddings_artigos)[0]

# Ordenar por similaridade
indices_ordenados = np.argsort(similaridades)[::-1]

# Formatação destacada para os resultados
recomendacoes = []
for i in range(5):
    idx = indices_ordenados[i]
    artigo = artigos[idx]
    similaridade = similaridades[idx]
    
    # Não recomendar artigos já vistos
    if artigo not in historico_usuario:
        recomendacoes.append(f"{len(recomendacoes)+1}. {similaridade:.3f} - {artigo}")

resultado_recomendacao = f"""📚 HISTÓRICO DO USUÁRIO:
{chr(10).join([f"• {i+1}. {artigo}" for i, artigo in enumerate(historico_usuario)])}

🎯 ARTIGOS DISPONÍVEIS:
{chr(10).join([f"• {i+1}. {artigo}" for i, artigo in enumerate(artigos)])}

📊 TOP 5 RECOMENDAÇÕES GERADAS:
{chr(10).join(recomendacoes)}

💡 COMO FUNCIONA:
1. Calculamos o embedding médio do histórico do usuário
2. Comparamos com embeddings de todos os artigos disponíveis
3. Ordenamos por similaridade (maior = mais relevante)
4. Filtramos artigos já vistos pelo usuário

🎯 RESULTADO:
O sistema recomendou artigos relacionados a Python e Data Science,
baseado no interesse demonstrado pelo usuário no histórico."""

print_resultado_destacado("EXEMPLO 4: SISTEMA DE RECOMENDAÇÃO", resultado_recomendacao, "success")


🎯 Sistema de Recomendação:

📚 Histórico do usuário:
1. Como aprender Python do zero
2. Data science com pandas

🔄 Gerando recomendações...

✅ ======================================================================
🟢                 EXEMPLO 4: SISTEMA DE RECOMENDAÇÃO                 🟢
✅ ======================================================================

📚 HISTÓRICO DO USUÁRIO:
• 1. Como aprender Python do zero
• 2. Data science com pandas

🎯 ARTIGOS DISPONÍVEIS:
• 1. Como aprender Python do zero
• 2. Introdução ao machine learning
• 3. Data science com pandas
• 4. Tutorial de deep learning
• 5. Como fazer análise de dados
• 6. Programação orientada a objetos
• 7. Estatística para data science
• 8. Visualização de dados com matplotlib

📊 TOP 5 RECOMENDAÇÕES GERADAS:
1. 0.406 - Visualização de dados com matplotlib
2. 0.303 - Programação orientada a objetos
3. 0.285 - Estatística para data science

💡 COMO FUNCIONA:
1. Calculamos o embedding médio do histórico do usuário
2. Comparamos co

### 🎯 **Exemplo 5: Clustering de Documentos**

![Classificação de Documentos](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/09.png)


In [10]:
# 🎯 Clustering de documentos por similaridade
documentos_cluster = [
    "Como fazer um bolo de chocolate",
    "Receita de bolo de cenoura",
    "Como programar em Python",
    "Tutorial de machine learning",
    "Como dirigir um carro",
    "Dicas para dirigir na estrada",
    "Como cuidar de plantas",
    "Jardinagem para iniciantes"
]

print("🎯 Clustering de Documentos:")
print("\n📚 Documentos:")
for i, doc in enumerate(documentos_cluster, 1):
    print(f"{i}. {doc}")

print("\n🔄 Agrupando documentos em clusters...")
clusters = helper.cluster_documents(documentos_cluster, n_clusters=3)

# Formatação destacada para os resultados
resultado_clustering = f"""📚 DOCUMENTOS ANALISADOS:
{chr(10).join([f"• {i+1}. {doc}" for i, doc in enumerate(documentos_cluster)])}

📊 RESULTADOS DO CLUSTERING:
"""

for cluster_id, docs in clusters.items():
    resultado_clustering += f"""
🔹 CLUSTER {cluster_id + 1} ({len(docs)} documentos):
{chr(10).join([f"   • {doc_info['documento']}" for doc_info in docs])}"""

resultado_clustering += f"""

💡 ANÁLISE DOS CLUSTERS:
• Cluster 1: Documentos sobre programação e tecnologia
• Cluster 2: Documentos sobre direção e transporte  
• Cluster 3: Documentos sobre culinária e jardinagem

🎯 COMO FUNCIONA:
1. Geramos embeddings para todos os documentos
2. Aplicamos algoritmo K-Means para agrupar por similaridade
3. Documentos com significado similar ficam no mesmo cluster
4. Cada cluster representa um tema ou categoria

✅ RESULTADO:
O sistema conseguiu agrupar automaticamente os documentos por temas,
mesmo sem conhecer previamente as categorias."""

print_resultado_destacado("EXEMPLO 5: CLUSTERING DE DOCUMENTOS", resultado_clustering, "success")


🎯 Clustering de Documentos:

📚 Documentos:
1. Como fazer um bolo de chocolate
2. Receita de bolo de cenoura
3. Como programar em Python
4. Tutorial de machine learning
5. Como dirigir um carro
6. Dicas para dirigir na estrada
7. Como cuidar de plantas
8. Jardinagem para iniciantes

🔄 Agrupando documentos em clusters...

✅ ======================================================================
🟢                EXEMPLO 5: CLUSTERING DE DOCUMENTOS                 🟢
✅ ======================================================================

📚 DOCUMENTOS ANALISADOS:
• 1. Como fazer um bolo de chocolate
• 2. Receita de bolo de cenoura
• 3. Como programar em Python
• 4. Tutorial de machine learning
• 5. Como dirigir um carro
• 6. Dicas para dirigir na estrada
• 7. Como cuidar de plantas
• 8. Jardinagem para iniciantes

📊 RESULTADOS DO CLUSTERING:

🔹 CLUSTER 3 (3 documentos):
   • Como fazer um bolo de chocolate
   • Receita de bolo de cenoura
   • Como dirigir um carro
🔹 CLUSTER 1 (2 documentos)

### 🎯 **Exemplo 6: RAG com Embeddings**

![RAG com Embeddings](https://raw.githubusercontent.com/pathbit/pathbit-academy-ai/refs/heads/master/0002_embeddings_vetorizacao/assets/10.png)


In [11]:
# 🔍 Simulação de RAG (Retrieval Augmented Generation)
# Este exemplo mostra como embeddings são usados para encontrar contexto relevante

# Base de conhecimento (simulando documentos)
base_conhecimento = [
    "Python é uma linguagem de programação interpretada, de alto nível e de propósito geral.",
    "Machine learning é um subcampo da inteligência artificial que se concentra em algoritmos que podem aprender com dados.",
    "Data science é um campo interdisciplinar que usa métodos científicos para extrair conhecimento de dados.",
    "Pandas é uma biblioteca Python para manipulação e análise de dados estruturados.",
    "NumPy é uma biblioteca Python fundamental para computação científica com arrays multidimensionais.",
    "Scikit-learn é uma biblioteca Python para machine learning que fornece ferramentas para classificação, regressão e clustering.",
    "Jupyter Notebook é um ambiente de desenvolvimento interativo que permite criar e compartilhar documentos com código, visualizações e texto."
]

# Pergunta do usuário
pergunta = "Como posso começar com análise de dados em Python?"

print("🔍 Sistema RAG - Retrieval Augmented Generation:")
print(f"Pergunta: '{pergunta}'")

print("\n📚 Base de Conhecimento:")
for i, doc in enumerate(base_conhecimento, 1):
    print(f"{i}. {doc}")

print(f"\n🔄 Buscando contexto relevante...")
resultados_rag = helper.search_documents(pergunta, base_conhecimento, top_k=3)

# Formatação destacada para os resultados
resultado_rag = f"""🔍 PERGUNTA DO USUÁRIO:
"{pergunta}"

📚 BASE DE CONHECIMENTO DISPONÍVEL:
{chr(10).join([f"• {i+1}. {doc}" for i, doc in enumerate(base_conhecimento)])}

📊 CONTEXTO RELEVANTE ENCONTRADO:
"""

for resultado in resultados_rag:
    resultado_rag += f"""
{resultado['posicao']}. SIMILARIDADE: {resultado['similaridade']:.3f}
   📄 DOCUMENTO: {resultado['documento']}
   {"✅ EXCELENTE MATCH" if resultado['similaridade'] > 0.6 else "⚠️ BOM MATCH" if resultado['similaridade'] > 0.4 else "❌ MATCH FRACO"}"""

resultado_rag += f"""

💡 COMO FUNCIONA O RAG:
1. 📝 Usuário faz uma pergunta
2. 🔍 Sistema busca documentos relevantes usando embeddings
3. 📊 Retorna os 3 documentos mais similares à pergunta
4. 🤖 LLM usa esse contexto para gerar uma resposta precisa

🎯 RESULTADO:
Com este contexto, um LLM poderia gerar uma resposta mais precisa!
Este é o princípio básico do RAG - encontrar informações relevantes primeiro.

✅ VANTAGENS DO RAG:
• Respostas baseadas em conhecimento específico
• Reduz alucinações do LLM
• Permite atualização da base de conhecimento
• Melhora a precisão das respostas"""

print_resultado_destacado("EXEMPLO 6: SISTEMA RAG", resultado_rag, "success")


🔍 Sistema RAG - Retrieval Augmented Generation:
Pergunta: 'Como posso começar com análise de dados em Python?'

📚 Base de Conhecimento:
1. Python é uma linguagem de programação interpretada, de alto nível e de propósito geral.
2. Machine learning é um subcampo da inteligência artificial que se concentra em algoritmos que podem aprender com dados.
3. Data science é um campo interdisciplinar que usa métodos científicos para extrair conhecimento de dados.
4. Pandas é uma biblioteca Python para manipulação e análise de dados estruturados.
5. NumPy é uma biblioteca Python fundamental para computação científica com arrays multidimensionais.
6. Scikit-learn é uma biblioteca Python para machine learning que fornece ferramentas para classificação, regressão e clustering.
7. Jupyter Notebook é um ambiente de desenvolvimento interativo que permite criar e compartilhar documentos com código, visualizações e texto.

🔄 Buscando contexto relevante...

✅ ===============================================

## 🎉 **Conclusão**

Parabéns! Você acabou de explorar os conceitos fundamentais de **embeddings e vetorização**. 

### 📚 **O que você aprendeu:**

1. **Conceitos Básicos** - Como texto é convertido em números
2. **Geração de Embeddings** - Usando modelos pré-treinados
3. **Cálculo de Similaridade** - Medindo semelhança entre textos
4. **Busca Semântica** - Encontrando documentos relevantes
5. **Sistema de Recomendação** - Aplicação prática
6. **Clustering** - Agrupando textos por similaridade
7. **RAG** - Base para sistemas inteligentes

### 💡 **Dica Final:**

> Embeddings são a base para muitos sistemas de IA modernos. Dominar este conceito é fundamental para construir soluções inteligentes que realmente entendem o significado do texto.

**Continue explorando e construindo! 🚀**
